# unknown genes

This is a text cell. Start editing!

In [1]:
import org.apache.spark._
import org.apache.spark.sql.types._
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._
import kernel.display.html
import org.bdgenomics.adam.rdd.ADAMContext._

In [2]:
def process(df: DataFrame) = df.select("uniref90", "liver", "kidney").as[(String, Double, Double)]
  .groupBy($"uniref90").agg(sum($"liver").as("liver"),sum($"kidney").as("kidney")).orderBy($"liver".desc)
def process_liver(df: DataFrame) = df.select("uniref90", "liver").as[(String, Double)]
  .groupBy($"uniref90").agg(sum($"liver").as("liver")).orderBy($"liver".desc)
def process_kidney(df: DataFrame) = df.select("uniref90", "kidney").as[(String, Double)].groupBy($"uniref90").agg(sum($"kidney").as("kidney")).orderBy($"kidney".desc)

In [3]:
val discovered = spark.readTSV("/data/results/gray-whale/Un-annotated/discovered.tsv", true)
discovered.show(10,1000)

+-------------------+------------------+-------------------------+-----------------------+---------------------------------------------------------------------------------------------------------------------+-------------+----------+-------------------------+------+------+-----------------+---------------------------+--------+---------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+----------+------------------+
|           uniref90|unknown_transcript|               transcript|                   type|                                                                                                                   go|   uniprot_ac|     taxon|               uniprot_id|entrez|refSeq|               gi|           

In [4]:
val disc_uniref = process(discovered)
disc_uniref.show(100,1000)

+-------------------+------------------+------------------+
|           uniref90|             liver|            kidney|
+-------------------+------------------+------------------+
|UniRef90_A0A287B1E2|       1919.685425|        913.876522|
|    UniRef90_G9L104|        527.380332|        363.998226|
|UniRef90_A0A1Z5LAU1|        471.968032|        224.839489|
|UniRef90_A0A212C9M8|         83.569752|         43.040916|
|UniRef90_A0A0P6IZM2|         66.219457|        128.537132|
|UniRef90_A0A1A6H5H6|           64.3221|         30.491912|
|    UniRef90_F6TX43|          62.56226|         19.479243|
|UniRef90_A0A075B7D9|         37.030672|           23.0172|
|UniRef90_A0A287D3X0|         33.208358|         20.814596|
|    UniRef90_Q2KJI6|         31.805498|         25.892438|
|UniRef90_A0A1A6HCL6|         25.962118|         12.170788|
|    UniRef90_L8E8V9|         23.147446|         40.208854|
|    UniRef90_G1SLK5|         22.939142|         69.150378|
|    UniRef90_G3I7K1|         17.598159|

Expressions
-----------

In [6]:
val exp = "/data/results/gray-whale/Expressions/uniref90/"
val bowhead_liver_uni = spark.readTSV(exp + "bowhead_alaska_uniref90_all.tab", true)
val bowhead_kidney_uni = spark.readTSV(exp + "bowhead_greenland_uniref90_all.tab", true)
val naked_mole_rat_uni = spark.readTSV(exp + "naked_mole_rat_uniref90_all.tab", true)
val gray_whale_uni = spark.readTSV(exp + "gray_whale_uniref90_all.tab", true)
val minke_uni = spark.readTSV(exp + "minky_whale_uniref90_all.tab", true)
minke_uni.show(10,1000)

+-------------------+--------------------------------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
allall_discoveredval ufs

+-----------------+------------+-----------+
|         uniref90|       liver|     kidney|
+-----------------+------------+-----------+
|  UniRef90_G7NBL7|41972.969113|  11.407494|
|  UniRef90_P68032|38942.087176|1095.007144|
|UniRef90_P05976-2|33553.032638|  11.981362|
|  UniRef90_J9NT19|17727.348262|   5.104173|
|  UniRef90_P00355|17238.197014|1909.346808|
|  UniRef90_F7BE76|15548.889057|   7.877229|
|  UniRef90_G5BUM2| 9223.149455|  15.045115|
|  UniRef90_G3U3B7| 8600.329817|6051.021132|
|  UniRef90_F6RTU1| 8245.044231|   12.59566|
|  UniRef90_P13412| 8243.894836|   2.708279|
+-----------------+------------+-----------+
only showing top 10 rows



In [8]:
val bowhead = bowhead_liver.join(bowhead_kidney, Seq("uniref90"), "outer")
bowhead.show(10,100)

+-------------------+-------+-------------------+
|           uniref90|  liver|             kidney|
+-------------------+-------+-------------------+
|UniRef90_A0A061I3I4|   null|           1.505533|
|UniRef90_A0A087X159|   null|                0.0|
|UniRef90_A0A091DGA5|   null|0.42072499999999996|
|UniRef90_A0A091DKP3|    0.0|           0.772682|
|UniRef90_A0A091E0T2|    0.0|               null|
|UniRef90_A0A096N803|   null|           0.118794|
|UniRef90_A0A0A8W4T6|   null|           0.884026|
|UniRef90_A0A0A8WBN6|   null|           3.036852|
|UniRef90_A0A146X1Y5|   null|           1.015653|
|UniRef90_A0A1A6HID7|2.79132|           1.014345|
+-------------------+-------+-------------------+
only showing top 10 rows



In [9]:
val non_model_kidney = spark.readTSV("/data/results/gray-whale/Expressions/Comparisons/uniref90_comparisons/outer_kidney_uniref90_graywhale_bowhead_minky_NMR.tsv", true)
val non_model_liver = spark.readTSV("/data/results/gray-whale/Expressions/Comparisons/uniref90_comparisons/outer_liver_uniref90_graywhale_bowhead_minky_NMR.tsv", true)
val non_model = non_model_liver.join(non_model_kidney, Seq("uniref90"), "outer")
non_model.show(10,1000)

+-------------------+----------------+-------------+-----------------+-----------+-----------------+-------------------+------------------+----------+
|           uniref90|gray_whale_liver|bowhead_liver|minke_whale_liver|  NMR_liver|gray_whale_kidney|     bowhead_kidney|minke_whale_kidney|NMR_kidney|
+-------------------+----------------+-------------+-----------------+-----------+-----------------+-------------------+------------------+----------+
|UniRef90_A0A061I3I4|             0.0|         null|             null|       null|         1.033552|           1.505533|              null|      null|
|UniRef90_A0A087X159|            null|         null|              0.0|   6.754005|             null|                0.0|               0.0|  0.778651|
|UniRef90_A0A091DGA5|            null|         null|             null|       null|             null|0.42072499999999996|              null|      null|
|UniRef90_A0A091DKP3|        2.609368|          0.0|             null|       null|         1.4

In [10]:
val non_model_discovered = disc_uniref
.withColumnRenamed("liver", "gray_whale_liver").withColumnRenamed("kidney", "gray_whale_kidney")
.join(non_model, Seq("uniref90", "gray_whale_liver", "gray_whale_kidney"), "left_outer").orderBy($"gray_whale_liver".desc,$"gray_whale_kidney".desc)
non_model_discovered.show(100,1000)

+-------------------+------------------+------------------+-----------------+-----------------+---------+--------------+------------------+----------+
|           uniref90|  gray_whale_liver| gray_whale_kidney|    bowhead_liver|minke_whale_liver|NMR_liver|bowhead_kidney|minke_whale_kidney|NMR_kidney|
+-------------------+------------------+------------------+-----------------+-----------------+---------+--------------+------------------+----------+
|UniRef90_A0A287B1E2|       1919.685425|        913.876522|             null|             null|     null|   3937.762936|              null|      null|
|    UniRef90_G9L104|        527.380332|        363.998226|             null|             null|     null|          null|              null|      null|
|UniRef90_A0A1Z5LAU1|        471.968032|        224.839489|          205.938|             null|     null|   7780.302806|              null|      null|
|UniRef90_A0A212C9M8|         83.569752|         43.040916|          17.0402|             null

In [11]:
val human_u = spark.readTSV("/data/results/gray-whale/Expressions/uniref90/established/human_uniref90.tsv", true)
val mouse_u = spark.readTSV("/data/results/gray-whale/Expressions/uniref90/established/mouse_uniref90.tsv", true)
val cow_u = spark.readTSV("/data/results/gray-whale/Expressions/uniref90/established/cow_uniref90.tsv", true)


In [12]:
val all_discovered = non_model_discovered.join(human_u, Seq("uniref90"), "left_outer")
.join(cow_u, Seq("uniref90"), "left_outer").join(mouse_u, Seq("uniref90"), "left_outer")
.orderBy($"gray_whale_liver".desc).cache()
all_discovered.show(10,1000)

+-------------------+----------------+-----------------+-------------+-----------------+---------+--------------+------------------+----------+-----------+------------+---------+----------+-----------+------------+
|           uniref90|gray_whale_liver|gray_whale_kidney|bowhead_liver|minke_whale_liver|NMR_liver|bowhead_kidney|minke_whale_kidney|NMR_kidney|human_liver|human_kidney|cow_liver|cow_kidney|mouse_liver|mouse_kidney|
+-------------------+----------------+-----------------+-------------+-----------------+---------+--------------+------------------+----------+-----------+------------+---------+----------+-----------+------------+
|UniRef90_A0A287B1E2|     1919.685425|       913.876522|         null|             null|     null|   3937.762936|              null|      null|       null|        null|     null|      null|       null|        null|
|    UniRef90_G9L104|      527.380332|       363.998226|         null|             null|     null|          null|              null|      nu

In [13]:
val mapping = spark.readTSV("/data/indexes/uniprot/idmapping_selected.tab").toDF("UniProtKB-AC","UniProtKB-ID","Entrez","RefSeq","GI","PDB","GO",
"UniRef100","UniRef90","UniRef50","UniParc","PIR",
"NCBI-taxon","MIM","UniGene","PubMed",
"EMBL","EMBL-CDS","Ensembl","Ensembl_TRS","Ensembl_PRO","Additional PubMed"
    )
mapping.limit(20).show(20, 1000)

+------------+------------+-------+-----------+-------------------------------+----+----------------------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|UniProtKB-AC|UniProtKB-ID| Entrez|     RefSeq|                             GI| PDB|                                GO|       UniRef100|       UniRef90|       UniRef50|      UniParc| PIR|NCBI-taxon| MIM|UniGene|            PubMed|    EMBL|  EMBL-CDS|Ensembl|Ensembl_TRS|Ensembl_PRO|Additional PubMed|
+------------+------------+-------+-----------+-------------------------------+----+----------------------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|      Q6GZX4|  001R_FRG3G|2947773|YP_031579.1|             81941549; 49237298|null|             

In [14]:
(all_discovered.select("uniref90").count, all_discovered.select("uniref90").distinct.count)

(79,79)

In [15]:
all_discovered.writeTSV("/data/results/gray-whale/Un-annotated/discovered_uniref90_expressions.tsv")

parts of /data/results/gray-whale/Un-annotated/discovered_uniref90_expressions.tsv merged!


/data/results/gray-whale/Un-annotated/discovered_uniref90_expressions.tsv

In [16]:
all_discovered.count()

79

In [17]:
val an =all_discovered.select($"uniref90".as("Uniref90")).join(mapping.select("Uniref90", "UniProtKB-ID", "NCBI-taxon"), Seq("Uniref90"), "left_outer")
.distinct.orderBy($"Uniref90").cache()
an.show(10,100)

+-------------------+----------------+----------+
|           Uniref90|    UniProtKB-ID|NCBI-taxon|
+-------------------+----------------+----------+
|UniRef90_A0A075B7D9|A0A075B7D9_HUMAN|      9606|
|UniRef90_A0A075B7D9|A0A2I3SIU0_PANTR|      9598|
|UniRef90_A0A075B7D9|A0A2K6P0I7_RHIRO|     61622|
|UniRef90_A0A075B7D9|    F7H4I1_CALJA|      9483|
|UniRef90_A0A075B7D9|A0A2J8RCM8_PONAB|      9601|
|UniRef90_A0A075B7D9|A0A2K5P538_CERAT|      9531|
|UniRef90_A0A075B7D9|A0A2I3MBZ7_PAPAN|      9555|
|UniRef90_A0A075B7D9|A0A2R9CLE2_PANPA|      9597|
|UniRef90_A0A075B7D9|A0A2K5XZK2_MANLE|      9568|
|UniRef90_A0A075B7D9|    G1RV69_NOMLE|     61853|
+-------------------+----------------+----------+
only showing top 10 rows



In [18]:
(an.select("Uniref90").distinct.count)

79

In [19]:
val con = new ConcatenateString(";")
val anno = an.groupBy("Uniref90").agg(con($"UniProtKB-ID").as("proteins_in_the_cluster"), count($"UniProtKB-ID"), con($"NCBI-taxon").as("taxons_in_the_cluster"), count($"NCBI-taxon"))
anno.show(10,1000)

+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-----------------------------------------------------------------------------------------+-----------------+
|           Uniref90|                                                                                                                                                                                                                                             proteins_in_the_cluster|count(UniProtKB-ID)|                                                                    taxons_in_the_cluster|count(NCBI-taxon)|
+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
anno.distinct.count()

79

In [21]:
val discovered_details = anno.join(all_discovered.withColumnRenamed("uniref90","Uniref90"), Seq("Uniref90")).orderBy($"gray_whale_liver".desc)
discovered_details.show(10,100) 

+-------------------+----------------------------------------------------------------------------------------------------+-------------------+-----------------------------------------------------------------------------------------+-----------------+----------------+-----------------+-------------+-----------------+---------+--------------+------------------+----------+-----------+------------+---------+----------+-----------+------------+
|           Uniref90|                                                                             proteins_in_the_cluster|count(UniProtKB-ID)|                                                                    taxons_in_the_cluster|count(NCBI-taxon)|gray_whale_liver|gray_whale_kidney|bowhead_liver|minke_whale_liver|NMR_liver|bowhead_kidney|minke_whale_kidney|NMR_kidney|human_liver|human_kidney|cow_liver|cow_kidney|mouse_liver|mouse_kidney|
+-------------------+-------------------------------------------------------------------------------------------

In [22]:
discovered_details.writeTSV("/data/results/gray-whale/Un-annotated/discovered_uniref90_expressions_detailed.tsv")

parts of /data/results/gray-whale/Un-annotated/discovered_uniref90_expressions_detailed.tsv merged!


/data/results/gray-whale/Un-annotated/discovered_uniref90_expressions_detailed.tsv

In [23]:
(discovered_details.count(), discovered.count())

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 505.0 failed 1 times, most recent failure: Lost task 0.0 in stage 505.0 (TID 16575, localhost, executor driver): java.io.FileNotFoundException: File file:/data/results/gray-whale/Un-annotated/discovered.tsv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:132)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:177)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:101)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithoutKey_